In [ ]:
import re
import time
import pickle
import requests
import subprocess

In [ ]:
# Question 1

!uv --version

In [ ]:
# Question 2

with open("uv.lock") as f:
    content = f.read()

package_block = re.search(
    r'\[\[package\]\]\s+name\s*=\s*"scikit-learn"([\s\S]*?)(?=\[\[package\]\]|$)',
    content
)

if package_block:
    hash_match = re.search(r'sha256:[a-f0-9]+', package_block.group(1))
    if hash_match:
        print(hash_match.group(0))
    else:
        print("No hash found in scikit-learn block")
else:
    print("scikit-learn not found in uv.lock")

In [ ]:
# Model Load

url = "https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin"

response = requests.get(url)
response.raise_for_status()  # ensure it downloaded OK

dv, model = pickle.loads(response.content)

In [ ]:
# Question 3

record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

X = dv.transform(record)

model.predict_proba(X)[0, 1]

In [ ]:
# Question 4

process = subprocess.Popen(
    ["python", "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "9696"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

url = "http://127.0.0.1:9696/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
predictions = response.json()

print(predictions)

In [ ]:
# Question 5

!docker pull agrigorev/zoomcamp-model:2025
!docker images agrigorev/zoomcamp-model:2025

In [ ]:
# Building the Docker image and starting the container

!docker build -t customer-scoring-service:latest .
!docker run -d -p 9696:9696 customer-scoring-service:latest

In [ ]:
# Question 6


url = url = "http://127.0.0.1:9696/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

In [ ]:
!for /f %i in ('docker ps -q') do docker stop %i